# Homework 3

Problem 1: Proposals.  Revisit this notebook.  I'd like to evaluate different proposals.  In the notebook we tried two different proposals, a sort of "global proposal" (uniform on -10 to 10) and a more local proposal (x + a normal distribution).  Part (a) For the second proposal type, is the convergence better if you change the scale of the normal distribution jump?  Part (b)  What if you change the jump to x +/- 0.001 with equal likelihood?  Convergence is better or worse?

Problem 2: State spaces.  Revisit this notebook.  I'd like you to build a sampler for the Poisson distribution with lambda = 5.  Challenge: the Poisson distribution has a state space which is discrete, not continuous (it's a distribution of counts; we use it frequently in wildlife count data).  Experiment a bit with the proposal distribution.